<a href="https://colab.research.google.com/github/Kwanseok-K/colorful_lights/blob/master/colorful_lights/code/Modeling/200923_LightGBM_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/My Drive/test/NS_2019.csv", encoding= 'cp949')
df.shape

(38300, 31)

In [6]:
df = df.dropna()
df.shape

(37368, 31)

In [7]:
df1 = df.drop(['p_code','m_code','date','hms', 'org_time', 'product','sale_m','ymd'],axis=1) 
df1 = pd.get_dummies(df1, columns=['month','day','wday','hour','cloud','order','max_order','before','add'])
df1.head()

,rain,temp,primetime,FlowSubway,FlowCar,duration,category,price,total,Y,holiday,stop,CPI,Shop_Online,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,...,add_세탁기,add_소파,add_속옷_상의,add_속옷_세트,add_속옷_하의,add_수납,add_식기,add_신발,add_아우터,add_악세사리,add_안경류,add_에어컨,add_영양제,add_욕실,add_원피스,add_육류,add_음료,add_의류_상의,add_의류_세트,add_의류_하의,add_의류관리기,add_의자,add_작물,add_장롱,add_조리보조,add_주방,add_주방_기타,add_주방가전,add_책상,add_청소,add_청소기,add_침구,add_침대,add_카페트,add_커튼,add_케어,add_팬,add_포트,add_해산물,add_헤어
0,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,2099000.0,2.630326,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,4371000.0,5.477444,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,3262000.0,4.087719,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,6955000.0,8.715539,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,6672000.0,8.360902,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
df1=df1.assign(class_="")
splitt = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=2020)
grp = df['category']

In [9]:
for train_index, test_index in splitt.split(df1, grp):
    strat_train = df1.iloc[train_index,]
    strat_test = df1.iloc[test_index,]

In [10]:
# 카테고리 더미화 
strat_train = pd.get_dummies(strat_train, columns=['category'])
strat_test = pd.get_dummies(strat_test,  columns=['category'])

In [4]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=38b053d9656096e5b3f1e95a2dfdd50f3d4670c9e57195b5e6bed4535e2dd61d
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [11]:
print(strat_train.shape)
print(strat_test.shape)

(28026, 191)
(9342, 191)


In [12]:
q1=strat_train['price'].quantile(q=0.25, interpolation='nearest')
q3=strat_train['price'].quantile(q=0.75, interpolation='nearest')

In [13]:
strat_train['class_'].loc[strat_train['price'] <q1]=1
strat_train['class_'].loc[(strat_train['price']>=q1) & (strat_train['price'] <q3)]=2 
strat_train['class_'].loc[strat_train['price']>=q3]=3

strat_test['class_'].loc[strat_test['price'] <q1]=1
strat_test['class_'].loc[(strat_test['price']>=q1) & (strat_test['price'] <q3)]=2 
strat_test['class_'].loc[strat_test['price']>=q3]=3

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Outlier 제거 코드(일단 안 씀)

In [ ]:
def remove_outlier(data, column):
  fraud_column_data = data[column]
  quan_25 = np.percentile(fraud_column_data.values, 25)
  quan_75 = np.percentile(fraud_column_data.values, 75)

  iqr = (quan_75-quan_25) * 3
  lowest = quan_25 - iqr
  highest = quan_75 + iqr
  outlier_index = fraud_column_data[(fraud_column_data<lowest)|(fraud_column_data>highest)].index
  data.drop(outlier_index, axis = 0, inplace = True)
  print(data.shape)

  return data

In [ ]:
strat_train_copy = remove_outlier(strat_train, 'Y')

In [ ]:
strat_train = remove_outlier(strat_train , 'Y')

Low

In [17]:
low_train=strat_train[strat_train['class_']==1]
low_test=strat_test[strat_test['class_']==1]

In [18]:
low_train = low_train.drop(['class_','total'],axis =1)
low_test = low_test.drop(['class_','total'],axis =1)

In [19]:
low_train = low_train[low_train.Y!=0]
low_test = low_test[low_test.Y!=0]

In [22]:
X_train = low_train[low_train.columns.difference(['Y'])]
y_train = np.log(low_train['Y'])
X_test = low_test[low_test.columns.difference(['Y'])]
y_test = np.log(low_test['Y'])

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6097, 188) (6097,)
(2011, 188) (2011,)


In [23]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [30]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

In [25]:
 pbounds = {'learning_rate' : (0.01, 0.3),
            'n_estimators': (2000, 4000),
            'num_leaves' : (30, 150),
            'max_depth' : (-1, 15),
            'colsample_bytree' : (0.7, 1),
            'subsample' : (0.5, 0.8), 
            'min_child_samples': (20, 100),
            'subsample_for_bin': (20000, 100000),
            'reg_lambda' : (0.5,0.9), 'reg_alpha': (0.5,1.0)}

In [27]:
def lgb_hyper_param(learning_rate, n_estimators, max_depth, 
                        subsample, colsample_bytree, num_leaves, 
                        min_child_samples, subsample_for_bin,
                      reg_alpha, reg_lambda):
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    min_child_samples = int(min_child_samples)
    num_leaves = int(num_leaves)
    subsample_for_bin = int(subsample_for_bin)

    
    model = lgb.LGBMRegressor(learning_rate=learning_rate, n_estimators=n_estimators,
                     max_depth=max_depth, subsample= subsample,
                     colsample_bytree=colsample_bytree, num_leaves = num_leaves,
                    min_child_samples=min_child_samples, subsample_for_bin=subsample_for_bin,
                    reg_alpha=reg_alpha, reg_lambda = reg_lambda)
    
    model.fit(X_train, y_train)
    
    
    y_pred = model.predict(X_test)
    
    #mse = mean_squared_error(y_test, y_pred)
    #r2 = r2_score(y_test, y_pred)
    mape =  - mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [28]:
optimizer = BayesianOptimization(
    f=lgb_hyper_param,
    pbounds=pbounds,
    random_state=2020,
)

In [31]:
optimizer.maximize(init_points=3, n_iter=30, acq='ei')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -11.8     |  0.7423   |  0.2296   |  10.78    |  48.45    |  2.682e+0 |  110.0    |  0.6086   |  0.7246   |  0.5373   |  4.558e+0 |
|  2        | -11.83    |  0.986    |  0.04983  |  8.111    |  98.05    |  3.007e+0 |  110.1    |  0.5171   |  0.6824   |  0.5468   |  5.808e+0 |
|  3        | -11.24    |  0.7509   |  0.2699   |  4.974    |  50.38    |  3.717e+0 |  107.5    |  0.7917   |  0.7673   |  0.5533   |  8.794e+0 |
|  4        | -11.68    |  0.8327   |  0.2511   |  11.22    |  93.58    |  2.141e+0 |  48.74    |  0.8184   |  0.7223   |  0.5576   |  5.405e+0 |
|  5        | -11.73    |  0.854    |  0.08812  |  8.584    |  37.61    |  2.602e+0 |  35.81    |  0.7822   |  0.8744   |  0

In [32]:
optimizer.max

{'params': {'colsample_bytree': 0.8844377955345387,
  'learning_rate': 0.24173636670316062,
  'max_depth': 4.61414975399704,
  'min_child_samples': 40.70564222703128,
  'n_estimators': 3926.237273568207,
  'num_leaves': 132.46653989514022,
  'reg_alpha': 0.5987416135557838,
  'reg_lambda': 0.5173263271083249,
  'subsample': 0.6529860560241294,
  'subsample_for_bin': 89220.95362523853},
 'target': -10.860466242303177}

{'params': {'colsample_bytree': 0.8844377955345387,
  'learning_rate': 0.24173636670316062,
  'max_depth': 4.61414975399704,
  'min_child_samples': 40.70564222703128,
  'n_estimators': 3926.237273568207,
  'num_leaves': 132.46653989514022,
  'reg_alpha': 0.5987416135557838,
  'reg_lambda': 0.5173263271083249,
  'subsample': 0.6529860560241294,
  'subsample_for_bin': 89220.95362523853},
 'target': -10.860466242303177}

In [33]:
model = lgb.LGBMRegressor(learning_rate=optimizer.max['params']['learning_rate'], 
                          n_estimators=int(optimizer.max['params']['n_estimators']),
                          num_leaves = int(optimizer.max['params']['num_leaves']),
                          subsample = optimizer.max['params']['subsample'],
                          subsample_for_bin = int(optimizer.max['params']['subsample_for_bin']), 
                          random_state=2020,
                          colsample_bytree = optimizer.max['params']['colsample_bytree'],
                          min_child_samples = int(optimizer.max['params']['min_child_samples']),
                          max_depth = int(optimizer.max['params']['max_depth']),
                          reg_alpha = optimizer.max['params']['reg_alpha'], 
                          reg_lambda = optimizer.max['params']['reg_lambda'])

In [34]:
model.fit(X_train, y_train)
lgbm_pred = model.predict(X_test)
lgbm_pred_train = model.predict(X_train)

In [35]:
mape_train = mean_absolute_percentage_error(np.exp(y_train), np.exp(lgbm_pred_train))
mape_test = mean_absolute_percentage_error(np.exp(y_test), np.exp(lgbm_pred))

print('MAPE => train: %.2f,   test: %.2f' %(mape_train, mape_test))

MAPE => train: 14.10,   test: 29.00


Medium

In [42]:
med_train=strat_train[strat_train['class_']==2]
med_test=strat_test[strat_test['class_']==2]

In [43]:
med_train = med_train.drop(['class_','total'],axis =1)
med_test = med_test.drop(['class_','total'],axis =1)

In [44]:
med_train = med_train[med_train.Y!=0]
med_test = med_test[med_test.Y!=0]

In [45]:
print(med_train.shape)
print(med_test.shape)

(14791, 189)
(4975, 189)


In [46]:
X_train = med_train[med_train.columns.difference(['Y'])]
y_train = np.log(med_train['Y'])
X_test = med_test[med_test.columns.difference(['Y'])]
y_test = np.log(med_test['Y'])

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(14791, 188) (14791,)
(4975, 188) (4975,)


In [48]:
 pbounds1 = {'learning_rate' : (0.01, 0.3),
            'n_estimators': (2000, 4000),
            'num_leaves' : (30, 150),
            'max_depth' : (-1, 15),
            'colsample_bytree' : (0.7, 1),
            'subsample' : (0.5, 0.8), 
            'min_child_samples': (20, 100),
            'subsample_for_bin': (20000, 100000),
            'reg_lambda' : (0.5,0.9), 'reg_alpha': (0.5,1.0)}

In [49]:
def lgb_hyper_param1(learning_rate, n_estimators, max_depth, 
                        subsample, colsample_bytree, num_leaves, 
                        min_child_samples, subsample_for_bin,
                      reg_alpha, reg_lambda):
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    min_child_samples = int(min_child_samples)
    num_leaves = int(num_leaves)
    subsample_for_bin = int(subsample_for_bin)

    
    model = lgb.LGBMRegressor(learning_rate=learning_rate, n_estimators=n_estimators,
                     max_depth=max_depth, subsample= subsample,
                     colsample_bytree=colsample_bytree, num_leaves = num_leaves,
                    min_child_samples=min_child_samples, subsample_for_bin=subsample_for_bin,
                    reg_alpha=reg_alpha, reg_lambda = reg_lambda)
    
    model.fit(X_train, y_train)
    
    
    y_pred = model.predict(X_test)
    
    #mse = mean_squared_error(y_test, y_pred)
    #r2 = r2_score(y_test, y_pred)
    mape =  - mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [50]:
optimizer = BayesianOptimization(
    f=lgb_hyper_param1,
    pbounds=pbounds1,
    random_state=2020,
)

In [51]:
optimizer.maximize(init_points=3, n_iter=30, acq='ei')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -95.17    |  0.9959   |  0.2633   |  7.156    |  41.75    |  2.674e+0 |  56.03    |  0.6382   |  0.6373   |  0.7586   |  3.254e+0 |
|  2        | -92.1     |  0.7423   |  0.2296   |  10.78    |  48.45    |  2.682e+0 |  110.0    |  0.6086   |  0.7246   |  0.5373   |  4.558e+0 |
|  3        | -88.53    |  0.986    |  0.04983  |  8.111    |  98.05    |  3.007e+0 |  110.1    |  0.5171   |  0.6824   |  0.5468   |  5.808e+0 |
|  4        | -88.65    |  0.7887   |  0.1318   |  7.562    |  83.16    |  3.974e+0 |  142.2    |  0.5102   |  0.6505   |  0.7189   |  5.222e+0 |
|  5        | -94.74    |  0.9985   |  0.2822   |  8.82     |  43.87    |  3.947e+0 |  135.8    |  0.9332   |  0.8114   |  0

In [52]:
optimizer.max

{'params': {'colsample_bytree': 0.9610223217284577,
  'learning_rate': 0.24435226868650517,
  'max_depth': 3.9799770831020034,
  'min_child_samples': 99.42112225060113,
  'n_estimators': 3134.2424133892073,
  'num_leaves': 80.39202652725794,
  'reg_alpha': 0.7884486336576992,
  'reg_lambda': 0.8890709441502967,
  'subsample': 0.7668867344992205,
  'subsample_for_bin': 31225.01591263318},
 'target': -85.00897774430823}

{'params': {'colsample_bytree': 0.9610223217284577,
  'learning_rate': 0.24435226868650517,
  'max_depth': 3.9799770831020034,
  'min_child_samples': 99.42112225060113,
  'n_estimators': 3134.2424133892073,
  'num_leaves': 80.39202652725794,
  'reg_alpha': 0.7884486336576992,
  'reg_lambda': 0.8890709441502967,
  'subsample': 0.7668867344992205,
  'subsample_for_bin': 31225.01591263318},
 'target': -85.00897774430823}

In [53]:
model = lgb.LGBMRegressor(learning_rate=optimizer.max['params']['learning_rate'], 
                          n_estimators=int(optimizer.max['params']['n_estimators']),
                          num_leaves = int(optimizer.max['params']['num_leaves']),
                          subsample = optimizer.max['params']['subsample'],
                          subsample_for_bin = int(optimizer.max['params']['subsample_for_bin']), 
                          random_state=2020,
                          colsample_bytree = optimizer.max['params']['colsample_bytree'],
                          min_child_samples = int(optimizer.max['params']['min_child_samples']),
                          max_depth = int(optimizer.max['params']['max_depth']),
                          reg_alpha = optimizer.max['params']['reg_alpha'], 
                          reg_lambda = optimizer.max['params']['reg_lambda'])

In [54]:
model.fit(X_train, y_train)
lgbm_pred = model.predict(X_test)
lgbm_pred_train = model.predict(X_train)

In [55]:
mape_train = mean_absolute_percentage_error(np.exp(y_train), np.exp(lgbm_pred_train))
mape_test = mean_absolute_percentage_error(np.exp(y_test), np.exp(lgbm_pred))

print('MAPE => train: %.2f,   test: %.2f' %(mape_train, mape_test))

MAPE => train: 32.38,   test: 43.13


High

In [56]:
high_train=strat_train[strat_train['class_']==3]
high_test=strat_test[strat_test['class_']==3]

In [57]:
high_train = high_train.drop(['class_','total'],axis =1)
high_test = high_test.drop(['class_','total'],axis =1)

In [58]:
high_train = high_train[high_train.Y!=0]
high_test = high_test[high_test.Y!=0]

In [59]:
print(high_train.shape)
print(high_test.shape)

(5656, 189)
(1845, 189)


In [60]:
X_train = high_train[high_train.columns.difference(['Y'])]
y_train = np.log(high_train['Y'])
X_test = high_test[high_test.columns.difference(['Y'])]
y_test = np.log(high_test['Y'])

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5656, 188) (5656,)
(1845, 188) (1845,)


In [62]:
 pbounds2 = {'learning_rate' : (0.01, 0.3),
            'n_estimators': (2000, 4000),
            'num_leaves' : (30, 150),
            'max_depth' : (-1, 15),
            'colsample_bytree' : (0.7, 1),
            'subsample' : (0.5, 0.8), 
            'min_child_samples': (20, 100),
            'subsample_for_bin': (20000, 100000),
            'reg_lambda' : (0.5,0.9), 'reg_alpha': (0.5,1.0)}

In [63]:
def lgb_hyper_param2(learning_rate, n_estimators, max_depth, 
                        subsample, colsample_bytree, num_leaves, 
                        min_child_samples, subsample_for_bin,
                      reg_alpha, reg_lambda):
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    min_child_samples = int(min_child_samples)
    num_leaves = int(num_leaves)
    subsample_for_bin = int(subsample_for_bin)

    
    model = lgb.LGBMRegressor(learning_rate=learning_rate, n_estimators=n_estimators,
                     max_depth=max_depth, subsample= subsample,
                     colsample_bytree=colsample_bytree, num_leaves = num_leaves,
                    min_child_samples=min_child_samples, subsample_for_bin=subsample_for_bin,
                    reg_alpha=reg_alpha, reg_lambda = reg_lambda)
    
    model.fit(X_train, y_train)
    
    
    y_pred = model.predict(X_test)
    
    #mse = mean_squared_error(y_test, y_pred)
    #r2 = r2_score(y_test, y_pred)
    mape =  - mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [64]:
optimizer = BayesianOptimization(
    f=lgb_hyper_param2,
    pbounds=pbounds2,
    random_state=2020,
)

In [65]:
optimizer.maximize(init_points=3, n_iter=30, acq='ei')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -141.6    |  0.9959   |  0.2633   |  7.156    |  41.75    |  2.674e+0 |  56.03    |  0.6382   |  0.6373   |  0.7586   |  3.254e+0 |
|  2        | -143.0    |  0.7423   |  0.2296   |  10.78    |  48.45    |  2.682e+0 |  110.0    |  0.6086   |  0.7246   |  0.5373   |  4.558e+0 |
|  3        | -137.2    |  0.986    |  0.04983  |  8.111    |  98.05    |  3.007e+0 |  110.1    |  0.5171   |  0.6824   |  0.5468   |  5.808e+0 |
|  4        | -141.4    |  0.7887   |  0.1318   |  7.562    |  83.16    |  3.974e+0 |  142.2    |  0.5102   |  0.6505   |  0.7189   |  5.222e+0 |
|  5        | -140.2    |  0.9499   |  0.02346  |  12.4     |  43.76    |  2.177e+0 |  70.7     |  0.8094   |  0.8529   |  0

In [66]:
optimizer.max

{'params': {'colsample_bytree': 0.9538482424001378,
  'learning_rate': 0.047088127373434034,
  'max_depth': 4.946227392443182,
  'min_child_samples': 85.71579118441096,
  'n_estimators': 3306.9232274896613,
  'num_leaves': 66.71052458584967,
  'reg_alpha': 0.8634226851807956,
  'reg_lambda': 0.8334044312815695,
  'subsample': 0.7049497992030136,
  'subsample_for_bin': 50708.46245764874},
 'target': -131.11328083960842}

{'params': {'colsample_bytree': 0.9538482424001378,
  'learning_rate': 0.047088127373434034,
  'max_depth': 4.946227392443182,
  'min_child_samples': 85.71579118441096,
  'n_estimators': 3306.9232274896613,
  'num_leaves': 66.71052458584967,
  'reg_alpha': 0.8634226851807956,
  'reg_lambda': 0.8334044312815695,
  'subsample': 0.7049497992030136,
  'subsample_for_bin': 50708.46245764874},
 'target': -131.11328083960842}

In [67]:
model = lgb.LGBMRegressor(learning_rate=optimizer.max['params']['learning_rate'], 
                          n_estimators=int(optimizer.max['params']['n_estimators']),
                          num_leaves = int(optimizer.max['params']['num_leaves']),
                          subsample = optimizer.max['params']['subsample'],
                          subsample_for_bin = int(optimizer.max['params']['subsample_for_bin']), 
                          random_state=2020,
                          colsample_bytree = optimizer.max['params']['colsample_bytree'],
                          min_child_samples = int(optimizer.max['params']['min_child_samples']),
                          max_depth = int(optimizer.max['params']['max_depth']),
                          reg_alpha = optimizer.max['params']['reg_alpha'], 
                          reg_lambda = optimizer.max['params']['reg_lambda'])

In [68]:
model.fit(X_train, y_train)
lgbm_pred = model.predict(X_test)
lgbm_pred_train = model.predict(X_train)

In [69]:
mape_train = mean_absolute_percentage_error(np.exp(y_train), np.exp(lgbm_pred_train))
mape_test = mean_absolute_percentage_error(np.exp(y_test), np.exp(lgbm_pred))

print('MAPE => train: %.2f,   test: %.2f' %(mape_train, mape_test))

MAPE => train: 37.76,   test: 54.65


In [70]:
(29*2011 + 43.13*4975 + 54.65*1845) / (2011+4975+1845)

42.3191031593251